In [0]:
from pyspark.sql.functions import when,timestamp_diff, col
from datetime import date
from dateutil.relativedelta import relativedelta

In [0]:
## Il y a 2 mois, le 1er du mois
two_month_ago_start_date = date.today().replace(day=1) - relativedelta(months=-2)
## il y a 1 mois, le 1er
one_month_ago_end_date = date.today().replace(day=1) - relativedelta(months=-1)

In [0]:
df = spark.read.table("nyctaxi.01_bronze.yellow_trips_raw").filter(f"tpep_pickup_datetime >= '{two_month_ago_start_date}' and tpep_pickup_datetime < '{one_month_ago_end_date}'")

In [0]:
df = df.select(
    when(df.VendorID == 1, "Creative Mobile Technologies, LLC")
    .when(df.VendorID == 2, "Curb Mobility")
    .when(df.VendorID == 6, "Myle Technologies")
    .when(df.VendorID == 7, "Helix" )
    .otherwise("Unknown")
    .alias("vendor"),

    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "passenger_count",
    "trip_distance",
    timestamp_diff("MINUTE", col("tpep_pickup_datetime"), col("tpep_dropoff_datetime")).alias("trip_duration_mins"),

    when(df.RatecodeID==1, "Standard rate")
    .when(df.RatecodeID==2, "JFK")
    .when(df.RatecodeID==3, "Newark")
    .when(df.RatecodeID==4, "Nassau or Westchester")
    .when(df.RatecodeID==5, "Negotiated fare")
    .when(df.RatecodeID==6, "Group ride")
    .otherwise("Unknown")
    .alias("rate_type"),

    "store_and_fwd_flag",
    col("PULocationID").alias("pu_location_id"),
    col("DOLocationID").alias("do_location_id"),

    when(df.payment_type==1, "Credit card")
    .when(df.payment_type==2, "Cash")
    .when(df.payment_type==3, "No charge")
    .when(df.payment_type==4, "Dispute")
    .when(df.payment_type==5, "Unknown")
    .when(df.payment_type==6, "Voided trip")
    .otherwise("Unknown")
    .alias("payment_type"),

    col("Airport_fee").alias("airport_fee"),
    "cbd_congestion_fee",
    "fare_amount",
   "extra",
    "mta_tax",
    "tip_amount",
    "tolls_amount",
    "improvement_surcharge",
    "total_amount",
    "congestion_surcharge"
)

In [0]:
# display(df)

In [0]:
df.write.mode("append").saveAsTable("nyctaxi.02_silver.yellow_trips_cleansed")


In [0]:
# spark.read.table("nyctaxi.02_silver.yellow_trips_cleansed").display()
